In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pulp import *

In [ ]:
space_data = {
    # "name" : (x, y, area)
    "P1": (-56, -1298, 5.4),
    "P2": (-602, -1030, 7.52),
    "P3": (-394, -1042, 8),
    "P4": (-195, -1046, 8),
    "P5": (1, -1046, 7.56),
    "P6": (-814, -932, 4.19),
    "P7": (-824, -643, 6.28),
    "P8": (-600, -600, 7.6),
    "P9": (-399, -639, 8),
    "P10": (-195, -632, 8),
    "P11": (0, -600, 7.67),
    "P12": (56, -334, 5.4),
    "P13": (-195, -258, 5.98),
    "P14": (-417, -269, 7.34),
    "P15": (-794, -258, 7.97),
    "P16": (408, -158, 5.64),
    "P17": (189, -43, 6.11),
    "P18": (0, 0, 7.11), # <--------------------------------------- HQ
    "P19": (600, 0, 4.92),
    "P20": (-931, 75, 1.38),
    "P21": (-808, 75, 8),
    "P22": (-604, 79, 7.82),
    "P23": (-403, 136, 5.53),
    "P24": (-206, 133, 5.64),
    "P25": (800, 200, 5.05),
    "P26": (1000, 400, 4.75),
    "P27": (-927, 445, 1.28),
    "P28": (-801, 434, 6.64),
    "P29": (-597, 431, 6.54),
    "P30": (-349, 380, 6.76),
    "P31": (153, -492, 1.47),
}

In [ ]:
spaces = pd.DataFrame(space_data).T
spaces.columns = ["x", "y", "area"]
spaces.reset_index(inplace=True)
spaces.head()

In [ ]:
dist = []
for j in range(len(spaces)):
    temp = []
    temp.append(spaces.index[j])
    for k in range(len(spaces)):
        if (j != k):
            distance = np.linalg.norm(spaces.iloc[j, [1,2]] - spaces.iloc[k, [1,2]] + 0.000001) / 1000
        else:
            distance = 9999

        dist.append(distance)

In [ ]:
# Define the problem
tsp1 = LpProblem("Traveling Salesman Problem", LpMinimize)

# Define decision variables
optcr = 0.0001
i = list(range(1, 32))
j = list(range(1, 32))

x = LpVariable.dicts("x", [(i_val, j_val) for i_val in i for j_val in j], cat="Binary")
u = LpVariable.dicts("u", i, lowBound=0, cat="Continuous")
m = LpVariable("m", lowBound=0, cat="Integer")

# Define objective function
tsp1 += lpSum(x[(i, j)] * (dist[i-1][j-1]/10 + 0.5) for i in range(1, 32) for j in range(1, 32)), "Objective Function"

# Define constraints
tsp1 += lpSum(x[(i, j)] for i in range(1, 32) if i != 18 for j in range(1, 32) if j != 18) == 1, "r1"
tsp1 += lpSum(x[(i, j)] for i in range(1, 32) if i != 18 for j in range(1, 32) if j != 18) == 1, "r2"
for j_val in range(1, 32):
    if j_val != 18:
        tsp1 += lpSum(x[(i_val, j_val)] for i_val in range(1, 32)) == 1, f"r3_{j_val}"
        tsp1 += lpSum(x[(i_val, j_val)] for i_val in range(1, 32)) == 1, f"r4_{j_val}"
tsp1 += lpSum(x[(i, j)] * (dist[i-1][j-1]/10 + 0.5) for i in range(1, 32) for j in range(1, 32)) <= 8 * 3, "r5"
tsp1 += lpSum(dist[i-1][j-1] * x[(i, j)] for i in range(1, 32) for j in range(1, 32)) >= 0, "r6"

for i_val in range(1, 32):
    for j_val in range(1, 32):
        if i_val != 18 and j_val != 18 and i_val != j_val:
            tsp1 += u[i_val] - u[j_val] + 31 * x[(i_val, j_val)] <= 30, f"subtours_{i_val}_{j_val}"

# Solve the problem
tsp1.solve()

# Print the results
print("Status:", LpStatus[tsp1.status])
print("Objective Value:", value(tsp1.objective))
for v in tsp1.variables():
    print(v.name, "=", v.varValue)